## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

This first implementation will use a simple, brute-force type of RAG..

### Sidenote: Business applications of this week's projects

RAG is perhaps the most immediately applicable technique of anything that we cover in the course! In fact, there are commercial products that do precisely what we build this week: nuanced querying across large databases of information, such as company contracts or product specs. RAG gives you a quick-to-market, low cost mechanism for adapting an LLM to your business area.

In [31]:
import sys
sys.path.append('/run/media/sijanpaudel/New Volume/New folder/llm_engineering')

In [32]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
from LLMHandler import LLMHandler
from key_utils import get_next_key

In [2]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()

In [22]:
# With massive thanks to student Dr John S. for fixing a bug in the below for Windows users!

context = {}

employees = glob.glob("knowledge-base/employees/*")

for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [23]:
employees

['knowledge-base/employees/Alex Chen.md',
 'knowledge-base/employees/Alex Harper.md',
 'knowledge-base/employees/Alex Thomson.md',
 'knowledge-base/employees/Avery Lancaster.md',
 'knowledge-base/employees/Emily Carter.md',
 'knowledge-base/employees/Emily Tran.md',
 'knowledge-base/employees/Jordan Blake.md',
 'knowledge-base/employees/Jordan K. Bishop.md',
 'knowledge-base/employees/Maxine Thompson.md',
 'knowledge-base/employees/Oliver Spencer.md',
 'knowledge-base/employees/Samantha Greene.md',
 'knowledge-base/employees/Samuel Trenton.md']

In [24]:
context["Lancaster"]

"# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferenc

In [25]:
products = glob.glob("knowledge-base/products/*")

for product in products:
    name = product.split(os.sep)[-1][:-3]
    doc = ""
    with open(product, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [51]:
context.keys()

dict_keys(['Chen', 'Harper', 'Thomson', 'Lancaster', 'Carter', 'Tran', 'Blake', 'Bishop', 'Thompson', 'Spencer', 'Greene', 'Trenton', 'Carllm', 'Homellm', 'Markellm', 'Rellm'])

In [122]:
system_message = """
You are an expert assistant on Insurellm, the Insurance Tech company.
Answer questions **only using the provided context**.

- If the answer is present in the context, provide a brief, accurate, and helpful response.
- If the answer is **not in the context**, respond politely and naturally. 
  For example: 
  "I’m sorry, I don’t have information about that. Is there something else I can help you with?" 
  or 
  "I can’t provide details on that topic, but feel free to ask me about other information you need."
- Do **not** make up information or guess.
- Keep answers concise, courteous, and professional.
"""


In [123]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context          

In [124]:
get_relevant_context("Who is chen?")

['# HR Record\n\n# Alex Chen\n\n## Summary\n- **Date of Birth:** March 15, 1990  \n- **Job Title:** Backend Software Engineer  \n- **Location:** San Francisco, California  \n\n## Insurellm Career Progression\n- **April 2020:** Joined Insurellm as a Junior Backend Developer. Focused on building APIs to enhance customer data security.\n- **October 2021:** Promoted to Backend Software Engineer. Took on leadership for a key project developing a microservices architecture to support the company\'s growing platform.\n- **March 2023:** Awarded the title of Senior Backend Software Engineer due to exemplary performance in scaling backend services, reducing downtime by 30% over six months.\n\n## Annual Performance History\n- **2020:**  \n  - Completed onboarding successfully.  \n  - Met expectations in delivering project milestones.  \n  - Received positive feedback from the team leads.\n\n- **2021:**  \n  - Achieved a 95% success rate in project delivery timelines.  \n  - Awarded "Rising Star" 

In [125]:
get_relevant_context("Who is Avery and what is carllm?")

['# Product Summary\n\n# Carllm\n\n## Summary\n\nCarllm is an innovative auto insurance product developed by Insurellm, designed to streamline the way insurance companies offer coverage to their customers. Powered by cutting-edge artificial intelligence, Carllm utilizes advanced algorithms to deliver personalized auto insurance solutions, ensuring optimal coverage while minimizing costs. With a robust infrastructure that supports both B2B and B2C customers, Carllm redefines the auto insurance landscape and empowers insurance providers to enhance customer satisfaction and retention.\n\n## Features\n\n- **AI-Powered Risk Assessment**: Carllm leverages artificial intelligence to analyze driver behavior, vehicle conditions, and historical claims data. This enables insurers to make informed decisions and set competitive premiums that reflect true risk profiles.\n\n- **Instant Quoting**: With Carllm, insurance companies can offer near-instant quotes to customers, enhancing the customer exper

In [126]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [127]:
print(add_context("Who is avery??"))

Who is avery??


In [128]:
handler = LLMHandler(system_message=system_message)

In [129]:
# ==============================================================================
# 5. THE GRADIO WRAPPER FUNCTION (TEXT-ONLY)
#    This function connects the Gradio UI to your LLMHandler.
# ==============================================================================
def chat_wrapper(message, history):
    """
    This is the core function for the Gradio interface.

    Args:
        message (str): The latest user input from the chat text box.
        history (list): Gradio's conversation history. We ignore this because
                        LLMHandler manages its own internal history.
    """
    # --- STEP 1: Preprocess the user's message to add relevant context ---
    message_with_context = add_context(message)
    
    # Optional: Print to the console for debugging to see if context was added
    if message_with_context != message:
        print("--- Context was added to the prompt ---")
        print(message_with_context)
        print("---------------------------------------")
    else:
        print(f"--- No context found for: '{message}' ---")

    try:
        # --- STEP 2: Call the handler with the (potentially enhanced) message ---
        generator = handler.run(
            user_input=message_with_context,
            task_type="text",
            stream=True  # We must request a stream for the live response
        )
        
        # --- STEP 3: Stream the response back to Gradio ---
        # Gradio's chat interface expects the *entire accumulated response* on each yield.
        full_response = ""
        for chunk in generator:
            # The chunk from your handler is a LangChain AIMessageChunk object
            if chunk and hasattr(chunk, 'content'):
                full_response += chunk.content
                yield full_response
            
    except Exception as e:
        # Display any errors directly in the chat window
        yield f"⚠️ An error occurred: {str(e)}"

# ======================================================

In [130]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history
#     message = add_context(message)
#     messages.append({"role": "user", "content": message})

#     stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

#     response = ""
#     for chunk in stream:
#         response += chunk.choices[0].delta.content or ''
#         yield response

## Now we will bring this up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [131]:
# 4. Setup the Gradio Interface
# We also link the "Clear" button to reset the handler's internal history.
with gr.ChatInterface(fn=chat_wrapper,type="messages", title="Gemini-Powered Assistant") as demo:
    # When the user clicks the built-in Clear button, also clear the handler's memory
    demo.chatbot.clear(lambda: handler.clear_history())

demo.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


--- No context found for: 'Who is Avery?' ---
➡️ Attempt 1/11 using key from 'nepalcric4'...
